In [ ]:
import numpy as np
import pandas as pd
import os
import regex as re
import math


ama = pd.read_csv('./amazon.csv', index_col=0)

#drop rows with all NaN values
ama = ama.dropna(how='all').drop_duplicates()


#clean discounted price
temp = ['Discounted Price']
for i in temp:
    
    ama[i] = ama[i].str.extract(r'(\d+\.?\d+?)')
    ama[i] = ama[i].astype(float)*7.75
    ama[i] = ama[i].apply(lambda x: round(x, 0))
    ama[i] = ama[i].astype(str).map(lambda x: x.strip('.0')).replace('nan','0').astype(int)
    
    
#make new column for 'price range'

criteria = [ama['Discounted Price'].between(0,100),
                ama['Discounted Price'].between(101,200),
                ama['Discounted Price'].between(201,300),
                ama['Discounted Price'].between(301,400),
                ama['Discounted Price'].between(401,500),
                ama['Discounted Price'].between(501,600),
                ama['Discounted Price'].between(601,700),
                ama['Discounted Price'].between(801,900),
                ama['Discounted Price'].between(901,1000),
                ama['Discounted Price'].between(1001,10000)]

values = ['0-100', '101-200', '201-300', '301-400', '401-500',
          '501-600', '601-700', '701-800', '901-1000', '>1000']


ama['Price Range'] = np.select(criteria, values, 0)



#clean original price
temp = ['Original Price']
for i in temp:
    ama[i] = ama[i].str.extract(r'(\d+\.?\d+?)')
    ama[i] = ama[i].astype(float)*7.75
    ama[i] = ama[i].apply(lambda x: round(x, 0))
    ama[i] = ama[i].astype(str).map(lambda x: x.strip('.0')).replace('nan',0).astype(int)
    
    
        
#clean 'No. of Reviews'
temp = ['No. of Reviews']
for i in temp:
    ama[i] = ama[i].astype(str).map(lambda x: x.strip('.0')).replace('nan',0).astype(int).replace('0',np.nan)

       
#clean Ratings'
temp = ['Ratings']
for i in temp:

    ama[i] = ama[i].astype(float)
    
    
#clean product category
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Moisturizers' 
                                                     if (x!=np.nan and 'Balms & Moisturizers' in x) else x)   
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Moisturizers' 
                                                     if (x!=np.nan and 'Creams & Moisturizers' in x) else x)   
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Moisturizers' 
                                                     if (x!=np.nan and 'Moisturizers' in x) else x)   
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Hydrating Mask' 
                                                     if (x!=np.nan and 'Mask' in x) else x)   
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Sunscreen' 
                                                     if (x!=np.nan and 'Sunscreens' in x) else x)   
ama['Category'] =  ama['Category'].astype(str).apply(lambda x:'Cleanser' 
                                                     if (x!=np.nan and 'Cleanser' in x) else x)   

    
# clean the rest        
ama['Brand'] = ama['Brand'].astype(str).map(lambda x: x.replace('Brand: ','').lstrip(' '))
ama['Promotion Message'] = ama['Promotion Message'].astype(str).apply(lambda x:'Yes' if x!='nan' else 'No')


#add new column
ama['Platform'] = 'Amazon'
ama['Vendor'] = str('')
ama['Discount'] = ama['Original Price']-ama['Discounted Price']
ama['% Discount'] = ama['Discount']/ama['Original Price']
ama['% Discount'] = ama['% Discount'].apply(lambda x:x*100 if x>0 else 0)
ama['Sales'] = np.nan
ama['Revenue'] = np.nan
ama['Male Only'] = 'Male'

ama = ama.rename(columns={'Discounted Price':'Discount Price',
                    'Brand':'Product Brand',
                    'Promotion Message':'Promo Tag',
                    'Category':'Product Category',
                    'Ratings':'Rating',
                    'No. of Reviews':'Review',
                    })


pd.set_option('display.max_rows', ama.shape[0]+1) 
ama = ama.reset_index().drop(['index'], axis=1)

ama.to_csv('amazon_cleaned.csv')